In [59]:
# tensorboard --logdir logs 指令启动终端 监听6006端口 并在网页上显示

In [37]:
from datetime import datetime
import io
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt

In [38]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()


In [39]:
def preprocess_data(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255
#     x = tf.reshape(x, [-1, 28 * 28])
    y = tf.cast(y, dtype=tf.int32)
    return x, y

In [40]:
# 预处理数据 划分训练集和测试集
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.map(preprocess_data).shuffle(10000).batch(128).repeat(10)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess_data).shuffle(10000).batch(128)

In [41]:
# 查看第一批次数据
for x_sample, y_sample in db_train.take(1):
    print(x_sample)
    print(y_sample)

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

In [42]:
model = keras.Sequential([
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10) # 输出层
])

model.build(input_shape=[None, 28 * 28])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              multiple                  401920    
_________________________________________________________________
dense_7 (Dense)              multiple                  131328    
_________________________________________________________________
dense_8 (Dense)              multiple                  32896     
_________________________________________________________________
dense_9 (Dense)              multiple                  8256      
_________________________________________________________________
dense_10 (Dense)             multiple                  2080      
_________________________________________________________________
dense_11 (Dense)             multiple                  330       
Total params: 576,810
Trainable params: 576,810
Non-trainable params: 0
________________________________________________

In [43]:
# 创建输出日志 用于可视化
current_time = datetime.now().strftime('%Y%m%d-%H%M%S')
log_dir = 'logs/' + current_time
summary_writer = tf.summary.create_file_writer(logdir=log_dir)

In [44]:
# 取出训练数据中的第一张图片 用于测试
sample_img = next(iter(db_train))[0]
# [1, 784]
sample_img = sample_img[0]
# 将第二个维度(784那一维) 转化为 28 * 28 * 1 因为是黑白的,只有一个颜色通道
sample_img = tf.reshape(sample_img, [1, 28, 28, 1])
with summary_writer.as_default():
    tf.summary.image('Training sample:', sample_img, step=0)

In [45]:
def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid(images):
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title='name')
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i], cmap=plt.cm.binary)
  
  return figure

In [58]:
# 优化器
optimizer = keras.optimizers.Adam(0.001)


for step, (x, y) in enumerate(db_train):
   # [batch, 28, 28, 1] => [batch, 28 * 28]
    x = tf.reshape(x, [-1, 28 * 28])
    y = tf.one_hot(y, depth=10)

    with tf.GradientTape() as tap:
        y_pred = model(x)
        y_true = tf.cast(y, dtype=tf.int32)

        # 交叉熵
        loss_ce = tf.reduce_mean(keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=True))

    grads = tap.gradient(loss_ce, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if step % 100 == 0:
        print(step, 'loss:', float(loss_ce))
        with summary_writer.as_default():
            tf.summary.scalar('train-loss', float(loss_ce), step=step)

    # 测试集上测试泛化能力
    if step % 500 == 0:
        # 测试集上测试泛化能力
        total_correct, total_num = 0, 0
        for _, (x, y) in enumerate(db_test):
            # 还得在转换一下 否则得出的结果是[128, 10] 不是[128]的格式
            x = tf.reshape(x, (-1, 28 * 28))     
            y_pred = model(x)
            y_prob = tf.nn.softmax(y_pred)
            y_pred = tf.argmax(y_prob, output_type=tf.int32, axis=1)

            # 比对
            correct = tf.equal(y, y_pred)
            correct = tf.cast(correct, dtype=tf.int32)
            total_correct += tf.reduce_sum(correct)
            total_num += x.shape[0]
            
        print('【正确率 aucc = %f】' % (total_correct / total_num))
        test_imgs = x_test[:25]
        test_imgs = tf.reshape(test_imgs, [-1, 28, 28, 1])
        print(test_imgs.shape)
        with summary_writer.as_default():
            tf.summary.scalar('test-aucc:', total_correct / total_num, step=step)
            # 图片一张一张的输出
            tf.summary.image('test-one-by-one-img', test_imgs, max_outputs=25, step=step)
            
            # 图片合成之后在输出 把25张小图片合成5 * 5的大图片
            test_imgs = tf.reshape(test_imgs, [-1, 28, 28])
            fig = image_grid(test_imgs)
            tf.summary.image('test_imgs_25', plot_to_image(fig), step=step)
            
            


0 loss: 1.5573608875274658
【正确率 aucc = 0.439600】
(25, 28, 28, 1)
100 loss: 0.6294048428535461
200 loss: 0.444131463766098
300 loss: 0.44052398204803467
400 loss: 0.4169237017631531
500 loss: 0.3385460078716278
【正确率 aucc = 0.862100】
(25, 28, 28, 1)
600 loss: 0.25533097982406616
700 loss: 0.33047640323638916
800 loss: 0.31786808371543884
900 loss: 0.37006673216819763
1000 loss: 0.2759595811367035
【正确率 aucc = 0.866600】
(25, 28, 28, 1)
1100 loss: 0.32472914457321167
1200 loss: 0.28321900963783264
1300 loss: 0.34115663170814514
1400 loss: 0.30202150344848633
1500 loss: 0.2683541774749756
【正确率 aucc = 0.858600】
(25, 28, 28, 1)
1600 loss: 0.2492804378271103
1700 loss: 0.35969164967536926
1800 loss: 0.304564505815506
1900 loss: 0.4160808324813843
2000 loss: 0.3882664442062378
【正确率 aucc = 0.873600】
(25, 28, 28, 1)
2100 loss: 0.25992709398269653
2200 loss: 0.3613847494125366
2300 loss: 0.31123286485671997
2400 loss: 0.1751662939786911
2500 loss: 0.19821418821811676
【正确率 aucc = 0.875600】
(25, 28, 